# Making HTTP Requests

- The `requests` library simplifies HTTP interactions by abstracting raw HTTP details, making it ideal for DevOps automation tasks.
- It allows you to query web services, trigger CI/CD builds, manage cloud resources, and integrate with APIs like GitHub in a straightforward way.
- In this notebook, we'll demonstrate installing `requests`, performing GET and POST requests, inspecting response data, and customizing requests with parameters and headers.
- The `requests` library is a third-party package and must be installed in your active virtual environment.
- Use `pip install requests==2.32.2` to add it to your project (pinning the version here so that we all work with the same version, but in other projects you can omit the version to install the latest), and consider pinning its version in `requirements.txt`.

In [50]:
GITHUB_ENDPOINT = "https://api.github.com"
HTTPBIN_ENDPOINT = "https://httpbin.org"

## Making GET Requests with `requests.get()`

- The GET method retrieves data from a specified URL; it’s the most common HTTP request type.
- Key parameters include `url`, optional `params` for query strings, `headers` for custom HTTP headers, and `timeout` to avoid hanging requests.
- The returned `Response` object provides `.status_code`, `.headers`, `.text`, `.content`, and `.json()`, plus `.raise_for_status()` to handle HTTP errors.

In [53]:
import requests
import json

response = requests.get("https://api.github.com", timeout=10)

"""
print(f"Status code: {response.status_code}")
print(f"Content-type: {response.headers['content-type']}")
print(".text attribute:")
print(response.text)
print("\n.content attribute:")
print(response.content)
print("\n.json() attribute:")
print(response.json())

"""

data = response.json()
print("Avaiable endpoints:")
print(json.dumps(data, indent=2))

Avaiable endpoints:
{
  "current_user_url": "https://api.github.com/user",
  "current_user_authorizations_html_url": "https://github.com/settings/connections/applications{/client_id}",
  "authorizations_url": "https://api.github.com/authorizations",
  "code_search_url": "https://api.github.com/search/code?q={query}{&page,per_page,sort,order}",
  "commit_search_url": "https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}",
  "emails_url": "https://api.github.com/user/emails",
  "emojis_url": "https://api.github.com/emojis",
  "events_url": "https://api.github.com/events",
  "feeds_url": "https://api.github.com/feeds",
  "followers_url": "https://api.github.com/user/followers",
  "following_url": "https://api.github.com/user/following{/target}",
  "gists_url": "https://api.github.com/gists{/gist_id}",
  "hub_url": "https://api.github.com/hub",
  "issue_search_url": "https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}",
  "issues_url": "https://a

## Passing URL Parameters with `params`

- Query parameters are passed as a dictionary to the `params` argument, and `requests` handles URL-encoding automatically.
- This makes it easy to filter, sort, or paginate API results without manually constructing the query string.
- You can inspect the final URL via `response.url` to confirm your parameters were applied correctly.

## Making POST Requests with `requests.post()`

- Use `requests.post()` to send data to a server, choosing between `data=` for form-encoded bodies or `json=` for JSON payloads.
- Providing a dictionary to `json=` automatically serializes it and sets `Content-Type: application/json`.
- The response can be inspected similarly to GET responses, using `.status_code`, `.json()`, and error handling.

## Common Pitfalls & How to Avoid Them

- **Not setting timeouts** can cause scripts to hang indefinitely; always include a `timeout` value.
- **Ignoring HTTP errors** means you might assume success when a request failed; use `response.raise_for_status()`.
- **Using `data=` instead of `json=`** sends form-encoded data, which may be rejected by modern APIs expecting JSON.
- **Hardcoding secrets** in code is insecure; use environment variables (e.g., via `python-dotenv`) and pass them in headers.